# Загрузи наши датасеты от разных разметчиков

In [1]:
!unzip -q /content/trains_only_2.v1i.yolov8.zip -d trains_only_2

In [2]:
!unzip -q /content/trains_only.v1i.yolov8.zip -d trains_only_1

# Инициализируем необходимый функционал

In [3]:
import os

In [4]:
def parse_annotation_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_label, x, y, width, height = map(float, parts)
            annotations.append({'class': int(class_label), 'bbox': (x, y, width, height)})

    return annotations

In [5]:
def compare_label_counts(folder1, folder2):
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))

    label_counts1 = {}
    label_counts2 = {}

    for file1, file2 in zip(files1, files2):
        file_path1 = os.path.join(folder1, file1)
        file_path2 = os.path.join(folder2, file2)

        annotations1 = parse_annotation_file(file_path1)
        annotations2 = parse_annotation_file(file_path2)

        label_counts1[file1] = len(annotations1)
        label_counts2[file2] = len(annotations2)

    return label_counts1, label_counts2

In [6]:
def calculate_bbox_deviation(bbox1, bbox2):
    deviation = [bbox1[i] - bbox2[i] for i in range(4)]
    return deviation

In [7]:
def compare_bbox_values(folder1, folder2):
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))

    bbox_deviations = {}

    for file1, file2 in zip(files1, files2):
        file_path1 = os.path.join(folder1, file1)
        file_path2 = os.path.join(folder2, file2)

        annotations1 = parse_annotation_file(file_path1)
        annotations2 = parse_annotation_file(file_path2)

        if len(annotations1) != len(annotations2):
            print(f"Number of annotations mismatch between {file1} and {file2}")
            continue

        bbox_deviations[file1] = []

        for annot1, annot2 in zip(annotations1, annotations2):
            bbox_deviation = calculate_bbox_deviation(annot1['bbox'], annot2['bbox'])
            bbox_deviations[file1].append({'class': annot1['class'], 'bbox_deviation': bbox_deviation})

    return bbox_deviations

# Проверим различие в количестве размеченных объектов

In [8]:
folder_path1 = '/content/trains_only_1/trains_only.v1i.yolov8/train/labels'
folder_path2 = '/content/trains_only_2/trains_only_2.v1i.yolov8/train/labels'

In [9]:
label_counts1, label_counts2 = compare_label_counts(folder_path1, folder_path2)
print("Label Counts:")
print("Folder 1:", label_counts1)
print("Folder 2:", label_counts2)


Label Counts:
Folder 1: {'1.txt': 1, '10.txt': 2, '11.txt': 0, '12.txt': 1, '13.txt': 1, '14.txt': 1, '15.txt': 4, '16.txt': 1, '17.txt': 4, '18.txt': 1, '19.txt': 1, '2.txt': 1, '20.txt': 2, '21.txt': 1, '22.txt': 1, '23.txt': 1, '24.txt': 1, '25.txt': 1, '26.txt': 0, '3.txt': 1, '4.txt': 4, '5.txt': 2, '6.txt': 1, '7.txt': 4, '8.txt': 4, '9.txt': 1}
Folder 2: {'1.txt': 1, '10.txt': 1, '11.txt': 0, '12.txt': 1, '13.txt': 1, '14.txt': 1, '15.txt': 5, '16.txt': 1, '17.txt': 4, '18.txt': 1, '19.txt': 1, '2.txt': 1, '20.txt': 2, '21.txt': 1, '22.txt': 1, '23.txt': 1, '24.txt': 1, '25.txt': 1, '26.txt': 0, '3.txt': 1, '4.txt': 4, '5.txt': 2, '6.txt': 1, '7.txt': 7, '8.txt': 4, '9.txt': 1}


Как можно заметить есть некоторые различия в количестве размеченных объектов.

Как в случае:
* 15.txt
* 10.txt
* 7.txt

# Проверим различие в bbox объектов

In [10]:
bbox_deviations = compare_bbox_values(folder_path1, folder_path2)

print("\nDeviation in Bbox Values:")
for file_name, deviations in bbox_deviations.items():
    # print(f"\nFile: {file_name}")
    for deviation in deviations:
        if deviation['bbox_deviation'] == [0.0, 0.0, 0.0, 0.0]:
            continue
        print(f"\nFile: {file_name} Class: {deviation['class']}, Deviation: {deviation['bbox_deviation']}")

Number of annotations mismatch between 10.txt and 10.txt
Number of annotations mismatch between 15.txt and 15.txt
Number of annotations mismatch between 7.txt and 7.txt

Deviation in Bbox Values:

File: 12.txt Class: 0, Deviation: [-0.18437499999999998, 0.0, -0.36812500000000004, 0.0]

File: 16.txt Class: 0, Deviation: [-0.018125000000000002, -0.003750000000000031, -0.03624999999999998, 0.0068749999999999645]

File: 17.txt Class: 0, Deviation: [-0.08374999999999999, 0.0, -0.16812499999999997, 0.0]

File: 17.txt Class: 0, Deviation: [-0.514375, -0.03125, 0.07125, -0.026875000000000038]

File: 17.txt Class: 0, Deviation: [0.14875000000000005, 0.036250000000000004, -0.12250000000000003, 0.06562499999999999]

File: 17.txt Class: 0, Deviation: [0.198125, 0.03125, 0.0018749999999999878, 0.17625]

File: 18.txt Class: 0, Deviation: [-0.007500000000000007, -0.0018750000000000155, -0.03, 0.004374999999999962]

File: 23.txt Class: 0, Deviation: [-0.0068749999999999645, 0.0, -0.014374999999999916,

При попытке найти различия в bbox двух разметчиков мы можем сравнить **bbox** от двух человек и посмотреть в чем были различия и как они велики.

Т.к. **trains_only_1** размечал я сам, а **trains_only_2** мой коллега без опыта разметки, благодаря данному функционалу я могу проверить интересующие меня файлы где разметка отличается от моей.

Т.к. число классов отличается только у 3-х изображений, а отклонение в координатах **bbox** достаточно велико только у **12.txt 17.txt 3.txt 8.txt** можно сделать вывод о вполне неплохой разметке от моего коллеги.